# 1. 콜금리 수집
네이버 금융에서 콜글리 데이터 수집

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import html2text

def saveCallRate() :
    df = pd.DataFrame(columns=['date','close','diff','change'])

    # 페이지수 수기로 계산하여 반복문 설정
    for pageIndex in range(1, 652):
        url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(pageIndex)
        page = requests.get(url)

        soup = BeautifulSoup(page.content, 'html.parser')
        trs = soup.find('tbody').find_all('tr')

        for tr in trs :
            tds = tr.find_all("td")
            tmp = [tds[0].text.strip(),float(tds[1].text.strip()),float(tds[2].text.strip()),tds[3].text.strip()]
            df.loc[len(df)] = tmp
            #print(tmp)        
    df.to_csv('./data/market/callrate.csv', index=False)
    
saveCallRate()

# 2. 1개월전 데이터와 비교하여 라벨부여

In [49]:
import pandas as pd

def applyCallrateLable() :
    callrate = pd.read_csv('./data/market/callrate.csv')

    callrate['pre_date'] = ''
    callrate['pre_close'] = ''
    callrate['pre_change'] = ''
    callrate['lable'] = 0

    for i, row in callrate.iterrows() :
        if callrate['date'][i] >= '2005.01.01' and callrate['date'][i] <= '2017.12.31' :
            callrate['pre_date'][i] = callrate['date'][i-21]
            callrate['pre_close'][i] = callrate['close'][i-21]
            callrate['pre_change'][i] = row['close'] - callrate['close'][i-21]
            if callrate['pre_change'][i] > 0 :
                callrate['lable'][i] = 1
            elif callrate['pre_change'][i] < 0 :
                callrate['lable'][i] = -1

        if callrate['date'][i] == '2005.01.01' :
            break

    callrate.loc[callrate.date >= '2005.01.01'].loc[callrate.date <= '2017.12.31'].to_csv('./data/market/callrate_lable.csv', index=False)

/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.

In [50]:
import pandas as pd

pd.read_csv('./data/market/callrate_lable.csv')

,date,close,diff,change,pre_date,pre_close,pre_change,lable
0,2017.12.29,1.58,0.03,+ 1.94%,2018.01.30,1.49,0.09,1
1,2017.12.28,1.55,0.04,+ 2.65%,2018.01.29,1.49,0.06,1
2,2017.12.27,1.51,0.01,-0.66%,2018.01.26,1.49,0.02,1
3,2017.12.26,1.52,0.01,+ 0.66%,2018.01.25,1.49,0.03,1
4,2017.12.22,1.51,0.02,-1.31%,2018.01.24,1.49,0.02,1
5,2017.12.21,1.53,0.02,+ 1.32%,2018.01.23,1.49,0.04,1
6,2017.12.20,1.51,0.01,+ 0.67%,2018.01.22,1.49,0.02,1
7,2017.12.19,1.50,0.00,0.00%,2018.01.19,1.49,0.01,1
8,2017.12.18,1.50,0.01,-0.66%,2018.01.18,1.49,0.01,1
9,2017.12.15,1.51,0.01,+ 0.67%,2018.01.17,1.49,0.02,1
